	# Author: Alexander Staub
	## Last changed: 2023.03.21
	## Purpose: Accessing the Spotify API


# Next up to dos - 23.03.28

* Figure out the rate limits and how to deal with them - timer or rate extension - there is a gpt thread going on this at the moment
* You changed the .append to .concat as per the warning in the loop below - if it throws an error, make sure it is not due to that
* currently you are pulling back non-useful elements from spotify. Go over what you can access via spotify and what may be of use as DVs. then change the code accordingly
* with the first random sample, a number of songs were not matched. Think of a way to get all the songs that weren't matched into a seperate dataframe
* Run a check to compare wether the artist name = the artist pulled back from spotify. Create a seperate df for those non-matches
* Do the same for the musicbrainz data (once you have only the songs with the correct year)

In [1]:
import os
import spotipy # lightweight Python library for the Spotify Web API
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials # to access authorised Spotify data

In [2]:
# package for accessing the local environment variables
import os

# get the client id and client secret from the environment variables
client_id = os.environ.get('SPOTIPY_CLIENT_ID')
client_secret = os.environ.get('SPOTIPY_CLIENT_SECRET')

if not client_id or not client_secret:
    raise ValueError("Spotify API credentials not found in environment variables")

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

In [4]:
#required package

from pandas import read_csv

# loading in the csv file with the songs from billboard and musicbrainz

df_HH_mb_matched = read_csv('C:/R work/Research/music_data/data/interim_data/df_hh_and_mb_leftjoin_fuzzy.csv')

In [7]:
# draw a random observation from the dataframe, retaining only the first two columns 
random_100_df = df_HH_mb_matched.sample(100)[['Artist', 'Track']]



In [8]:
#create the song infor retreival function

# added a bit to the function to handle errors and retries
import time

#here the max retries is 3
def get_song_data(song_name, artist_name, max_retries=3):
    query = f'track:{song_name} artist:{artist_name}'
    retries = 0
    
    #while loop to retry the query if it fails n times (as defined by max_retries)
    while retries <= max_retries:
        try:
            results = sp.search(q=query, type='track', limit=1)
            
            if results['tracks']['items']:
                track = results['tracks']['items'][0]
                song_data = {
                    'song_name': track['name'],
                    'artist_name': track['artists'][0]['name'],
                    'album_name': track['album']['name'],
                    'release_date': track['album']['release_date'],
                    'popularity': track['popularity'],
                    'explicit': track['explicit'],
                    'duration_ms': track['duration_ms'],
                    'preview_url': track['preview_url']
                }
                return song_data
            else:
                return None
        
        except Exception as e:
            print(f"Error: {e}")
            retries += 1
            sleep_time = 2 ** retries
            print(f"Retrying ({retries}/{max_retries}) after {sleep_time} seconds")
            time.sleep(sleep_time)
    
    return None



In [11]:
# create an empty dataframe
results_df = pd.DataFrame(columns=["Track", "Artist", "song_name", "artist_name", "album_name", "release_date", "popularity", "explicit", "duration_ms", "preview_url"])

# loop over the random sample dataframe

for index, row in random_100_df.iterrows():
    song_name = row["Track"]
    artist_name = row["Artist"]

    song_data = get_song_data(song_name, artist_name)
    if song_data:
        song_data["Track"] = song_name
        song_data["Artist"] = artist_name

        # changed results.append to results_df.concat as per the errors in the previous code
        
        results_df = results_df.concat(song_data, ignore_index=True)
    else:
        print(f"Song not found: {song_name} by {artist_name}")



ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))